# Bad Grade, Good Experience

Recently I was asked a question I hadn't thought about before: 

> *As a student, did you ever get a bad grade on a programming assignment?* 

It has been a long time since college so I've forgotten many of my assignments, but there is one I do remember. It was something like this:

## The Concordance Assignment

> *Using the [`Snobol`](http://www.snobol4.org/) language, read lines of text from the standard input and print a *concordance*, which is an alphabetized list of words in the text, with the line number(s) where each word appears. Words with different capitalization (like "A" and "a") should be merged into one entry.*

After studying Snobol a bit, I realized that the intended solution was along these lines:

1. Create an empty `dict`, let's call it `entries`.
2. Read the lines of text (tracking the line numbers), split them into words, and build up the string of line numbers for each word with something like
`entries[word] += line_number + ", "`
3. Convert the table into a two-dimensional `array` where each row has the two columns `[word, line_numbers]`.
4. Write a function to sort the array alphabetically (`sort` was not built-in to Snobol).
5. Write a function to print the array.

That would probably be about 40 lines of code; an easy task, the main point of which was just to get familiar with a new and different programming language. But I noticed three interesting things about Snobol:

* There is an *indirection* operator, `$`, so the two statements `word = "A"; $word = "1"` are equivalent to `A = "1"`.
* Uninitialized variables are treated as the empty string, so `$word += "1"` works even if  we haven't seen `$word` before.
* When the program ends, the Snobol interpreter automatically
prints the values of
every variable, sorted alphabetically, as a debugging aid.

That means I can do away with the `dict` and the `array`, eliminating steps 1, 3, 4, and 5, and just do step 2! My entire program is as follows (to make it easier to understand I use Python syntax, except for Snobol's `$word` indirection:

In [1]:
program = """
for num, line in enumerate(input):
    for word in re.findall(r"\w+", line.upper()):
        $word += str(num) + ", "
"""

That's 3 lines, not 40, and it works! Actually, it wasn't quite perfect, because the post-mortem printout included the values of the variables `num, line,` and `word`. So, reluctantly, I had to increase the program's line count by 33%:

In [2]:
program = """
for num, line in enumerate(input):
    for word in re.findall(r"\w+", line.upper()):
        $word += str(num) + ", "
del num, line, word
"""

To prove it works, I'll write a mock Snobol/Python interpreter, which is really just a call to the Python interpreter, `exec`, except that it does three Snobol-ish things: (1) the `context` in which the program is executed is a `defaultdict(str)`, which means that variables default to the empty string. The context is pre-loaded with builtins. (2) Any `$word` in the program is rewritten as `_context[word]`, which enables
indirection. (3) After the `exec` completes, the user-defined variables (but not the built-in ones) and their values are printed.

In [3]:
from collections import defaultdict
import re

def snobol(program, data=''):
    """A Python interpreter with three Snobol-ish features:
    (1) variables default to '', (2) $word indirection, (3) post-mortem printout."""
    context = defaultdict(str, vars(__builtins__))
    context.update(_context=context, re=re, input=data.splitlines())
    builtins = set(context) # The builtin functions won't appear in the printout
    program = re.sub(r'\$(\w+)', r'_context[\1]', program) # "$word" => "_context[word]"
    exec(program, context)
    print('-' * 50)
    for name in sorted(context):
        if name not in builtins and not name.startswith('_'):
            print('{:10} = {}'.format(name, context[name]))

Now we can run my program on some data:

In [4]:
data = """
There she was just a-walkin' down the street, singin' "Do wah diddy diddy dum diddy do"
Snappin' her fingers and shufflin' her feet, singin' "Do wah diddy diddy dum diddy do"
She looked good (looked good), she looked fine (looked fine)
She looked good, she looked fine and I nearly lost my mind
"""

snobol(program, data)

--------------------------------------------------
A          = 1, 
AND        = 2, 4, 
DIDDY      = 1, 1, 1, 2, 2, 2, 
DO         = 1, 1, 2, 2, 
DOWN       = 1, 
DUM        = 1, 2, 
FEET       = 2, 
FINE       = 3, 3, 4, 
FINGERS    = 2, 
GOOD       = 3, 3, 4, 
HER        = 2, 2, 
I          = 4, 
JUST       = 1, 
LOOKED     = 3, 3, 3, 3, 4, 4, 
LOST       = 4, 
MIND       = 4, 
MY         = 4, 
NEARLY     = 4, 
SHE        = 1, 3, 3, 4, 4, 
SHUFFLIN   = 2, 
SINGIN     = 1, 2, 
SNAPPIN    = 2, 
STREET     = 1, 
THE        = 1, 
THERE      = 1, 
WAH        = 1, 2, 
WALKIN     = 1, 
WAS        = 1, 


Looks good to me! But sadly, the grader did not agree, complaining that my program was not extensible: what if I wanted to cover two or more files in one run? What if I wanted the output to have a slightly different format? I argued that [YAGNI](https://en.wikipedia.org/wiki/You_aren%27t_gonna_need_it), but the grader wasn't impressed so I got  points taken off.

# TFW you flunk AI

Here's another example that I had completely forgotten about until 2016, when I was cleaning out some old files and came across my old college transcript. It turns out that *I flunked an AI course!* (Or at least, didn't complete it.) This course was
offered by Prof. Richard Millward in the Cognitive Science program.  I certainly remember a lot of influential material from this class: we read David Marr, we read Winston's just-published *Psychology of Computer Vision*, we read a chapter from Duda and Hart which was then only a few years old. The things I learned in that course have stuck with me for decades, but one thing that didn't stick is that my transcript said I never completed the course! I'm not sure what happened. I did an independent study with Ulf Grenander that semester; maybe  I had to drop the AI course to fit that in. I don't know. 

So in both the concordance program and the Cognitive Science AI class, I had a great experience and I learned a lot, even if it wasn't well-reflected in official credit. The moral is: look for the good experiences, and don;t worry about the official credit.


